## Import packages

In [ ]:
!pip install datasets --user

In [27]:
from datasets import load_dataset
from transformers import AutoFeatureExtractor
from functools import partial

import numpy as np
import torch
import torch.nn as nn

import timm.models.vision_transformer
import os

from tqdm import tqdm
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from timm.models.vision_transformer import PatchEmbed, Block

## Food101 Dataset 

original train set: 75,750

original validation set: 25,250

total: 101,000

In [25]:
food101 = load_dataset('food101')

In [3]:
food101['train'][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=384x512>,
 'label': 6}

In [4]:
from tqdm import tqdm
import numpy as np

total_r = 0
total_g = 0
total_b = 0
total_pixels = 0

# Variables for standard deviation
sum_squared_r = 0
sum_squared_g = 0
sum_squared_b = 0

indices_to_drop = []

for i in tqdm(range(len(food101['train']))):
    img = np.array(food101['train'][i]['image'].resize((224, 224))) / 255.
    
    if img.shape == (224, 224, 3):
        total_r += img[:, :, 0].sum()
        total_g += img[:, :, 1].sum()
        total_b += img[:, :, 2].sum()
        
        # For standard deviation
        sum_squared_r += np.sum(np.square(img[:, :, 0]))
        sum_squared_g += np.sum(np.square(img[:, :, 1]))
        sum_squared_b += np.sum(np.square(img[:, :, 2]))

        total_pixels += 224 * 224
    else:
        indices_to_drop.append(i)

mean_r = total_r / total_pixels
mean_g = total_g / total_pixels
mean_b = total_b / total_pixels

# Compute std for each channel
std_r = np.sqrt((sum_squared_r / total_pixels) - (mean_r ** 2))
std_g = np.sqrt((sum_squared_g / total_pixels) - (mean_g ** 2))
std_b = np.sqrt((sum_squared_b / total_pixels) - (mean_b ** 2))

print(f"Mean RGB: {mean_r}, {mean_g}, {mean_b}")
print(f"Std RGB: {std_r}, {std_g}, {std_b}")

100%|██████████| 75750/75750 [05:07<00:00, 245.96it/s]

Mean RGB: 0.5449871888617703, 0.4434935563380693, 0.34361316599832514
Std RGB: 0.27093838406970966, 0.2734508551865403, 0.2780531622290323


In [5]:
len(indices_to_drop)

3

In [6]:
food101_mean = np.array([mean_r, mean_g, mean_b])
food101_std = np.array([std_r, std_g, std_b])

In [7]:
def transform(example_batch):
    """
    reshape the images into 224 * 224
    """
    inputs = {}
    
    pixel_values = []
    labels = []
    for i in range(len(example_batch['image'])):
        x = example_batch['image'][i]
        y = example_batch['label'][i]
        if np.array(x.resize((224, 224))).shape == (224, 224, 3):
            pixel_values.append(torch.tensor(((np.array(x.resize((224, 224))) / 255. - food101_mean) / food101_std), dtype = torch.float).permute(2, 0, 1))
            labels.append(y)

    inputs['pixel_values'] = pixel_values
    inputs['label'] = labels
    return inputs

In [8]:
np.array(food101['train'][0]['image'].resize((224, 224))).shape

(224, 224, 3)

In [9]:
processed_food101 = food101.with_transform(transform)

In [10]:
train_dataset = processed_food101['train']

In [11]:
indices_to_choose = list(set(range(len(train_dataset))) - set(indices_to_drop))
filtered_train_dataset = train_dataset.select(indices_to_choose)

In [12]:
filtered_train_dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 75747
})

In [13]:
filtered_train_dataset[18507]['pixel_values'].shape

torch.Size([3, 224, 224])

## Masked AutoEnocder 

Positional Embedding

In [14]:
def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """
    
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype = float)
    omega /= embed_dim /2.
    omega = 1. / 10000 ** omega # (D/2,)
    
    pos = pos.reshape(-1) # (M,)
    out = np.einsum('m, d->md', pos, omega) # (M, D/2), outer product
    
    emb_sin = np.sin(out) # (M, D/2)
    emb_cos = np.cos(out) # (M, D/2)
    emb = np.concatenate([emb_sin, emb_cos], axis = 1) # (M, D)
    return emb

In [15]:
def get_2d_sincos_pos_embed_from_grid(embed_dim, grid):
    assert embed_dim % 2 == 0
    
    # use half of dimensions to encode grid_h
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0]) # (H*W, D/2)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1]) # (H*W, D/2)
    
    emb = np.concatenate([emb_h, emb_w], axis = 1)
    return emb

In [16]:
def get_2d_sincos_pos_embed(embed_dim, grid_size, cls_token = False):
    """
    grid_size: int of the grid height and width
    return:
    pos_embed: [grid_size*grid_size, embed_dim] or [1+grid_size*grid_size, embed_dim] (w/ or w/o cls_token)
    """
    grid_h = np.arange(grid_size, dtype = np.float32)
    grid_w = np.arange(grid_size, dtype = np.float32)
    grid = np.meshgrid(grid_h, grid_w)
    grid = np.stack(grid, axis = 0)
    
    grid = grid.reshape([2, 1, grid_size, grid_size])
    pos_embed = get_2d_sincos_pos_embed_from_grid(embed_dim, grid)
    if cls_token:
        pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis = 0)
    return pos_embed

Model Structure

In [17]:
class MaskedAutoencoderViT(nn.Module):
    """
    Masked Autoencoder with VisionTransformer backbone
    """
    def __init__(self, img_size = 224, patch_size = 16, in_chans = 3,\
                 embed_dim = 1024, depth = 24, num_heads = 16,\
                decoder_embed_dim = 512, decoder_depth = 8, decoder_num_heads = 16,\
                mlp_ratio = 4., norm_layer = nn.LayerNorm, norm_pix_loss = False):
        super().__init__()
        
        # ---------------
        # MAE encoder specifics
        self.in_chans = in_chans
        self.patch_embed = PatchEmbed(img_size, patch_size, in_chans, embed_dim)
        num_patches = self.patch_embed.num_patches
        
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim), requires_grad = False) # fixed sin-cos embedding
        
        self.blocks = nn.ModuleList([
            Block(embed_dim, num_heads, mlp_ratio, qkv_bias = True, norm_layer = norm_layer)
            for i in range(depth)
        ])
        self.norm = norm_layer(embed_dim)
        # ---------------
        
        # ---------------
        # MAE decoder specifics
        self.decoder_embed = nn.Linear(embed_dim, decoder_embed_dim, bias = True)
        
        self.mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))
        
        self.decoder_pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, decoder_embed_dim), requires_grad = False)
        
        self.decoder_blocks = nn.ModuleList([
            Block(decoder_embed_dim, decoder_num_heads, mlp_ratio, qkv_bias = True, norm_layer = norm_layer)
            for i in range(decoder_depth)
        ])
        
        self.decoder_norm = norm_layer(decoder_embed_dim)
        self.decoder_pred = nn.Linear(decoder_embed_dim, patch_size**2 * in_chans, bias = True) # decoder to patch
        # ---------------
        
        self.norm_pix_loss = norm_pix_loss
        
        self.initialize_weights()
        
    def initialize_weights(self):
        # initilization
        # initialize (and freeze) pos_embed by sin-cos embedding
        pos_embed = get_2d_sincos_pos_embed(self.pos_embed.shape[-1], int(self.patch_embed.num_patches**.5), cls_token = True)
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))
        
        decoder_pos_embed = get_2d_sincos_pos_embed(self.decoder_pos_embed.shape[-1], int(self.patch_embed.num_patches**.5), cls_token = True)
        self.decoder_pos_embed.data.copy_(torch.from_numpy(decoder_pos_embed).float().unsqueeze(0))
        
        # initialize patch_embed like nn.Linear (instead of nn.Conv2d)
        w = self.patch_embed.proj.weight.data
        torch.nn.init.xavier_uniform_(w.view([w.shape[0], -1]))
        
        # timm's trunc_normal_(std=.02) is effectively normal_(std=.02) as cutoff is too big (2.)
        torch.nn.init.normal_(self.cls_token, std=.02)
        torch.nn.init.normal_(self.mask_token, std=.02)
        
        # initialize nn.Linear and nn.LayerNorm
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            # we use xavier_uniform following official JAX ViT:
            torch.nn.init.xavier_uniform_(m.weight)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
            
    def patchify(self, imgs):
        """
        imgs: (N, 3, H, W)
        x: (N, L, patch_size**2 * 3)
        """
        p = self.patch_embed.patch_size[0]
        
        assert imgs.shape[2] == imgs.shape[3] and imgs.shape[2] % p == 0
        h = w = imgs.shape[2] // p
        x = imgs.reshape(shape = (imgs.shape[0], self.in_chans, h, p, w, p))
        x = torch.einsum('nchpwq -> nhpwqc', x)
        x = x.reshape(shape = (imgs.shape[0], h * w, p**2 * self.in_chans))
        return x
    
    def unpatchify(self, x):
        """
        x: (N, L, patch_size**2 * 3)
        imgs: (N, 3, H, W)
        """
        p = self.patch_embed.patch_size[0]
        h = w = int(x.shape[1]**.5)
        assert h * w == x.shape[1]
        
        x = x.reshape(shape = (x.shape[0], h, w, p, p, self.in_chans))
        x = torch.einsum('nhwpqc -> nchpwq', x)
        imgs = x.reshape(shape = (x.shape[0], self.in_chans, h * p, h * p))
        return imgs
    
    def random_masking(self, x, mask_ratio):
        """
        Perform per-sample masking by per-sample shuffling.
        Per-sample shuffling is done by argsort random noise.
        x: [N, L, D], sequence
        """
        
        N, L, D = x.shape # batch, length, dim
        len_keep = int(L * (1 - mask_ratio))
        
        noise = torch.rand(N, L, device = x.device) # noise in [0, 1]
        
        # sort noise for each sample
        ids_shuffle = torch.argsort(noise, dim = 1) # ascend: small is kept, large is removed
        ids_restore = torch.argsort(ids_shuffle, dim = 1)
        
        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]
        x_masked = torch.gather(x, dim = 1, index = ids_keep.unsqueeze(-1).repeat(1, 1, D))
        
        # generate the binary mask: 0 is kept, 1 is removed
        mask = torch.ones([N, L], device = x.device)
        mask[:, :len_keep] = 0
        # unshuffle to get the binary mask
        mask = torch.gather(mask, dim = 1, index = ids_restore)
        
        return x_masked, mask, ids_restore
    
    def forward_encoder(self, x, mask_ratio):
        # embed patches
        x = self.patch_embed(x)
        
        # add pos embed w/o cls token
        x = x + self.pos_embed[:, 1:, :]
        
        # masking: length -> length * mask_ratio
        x, mask, ids_restore = self.random_masking(x, mask_ratio)
        
        # append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim = 1)
        
        # apply Transformer blocks
        for blk in self.blocks:
            x = blk(x)
            
        x = self.norm(x)
        
        return x, mask, ids_restore
        
    def forward_decoder(self, x, ids_restore):
        # embed tokens
        x = self.decoder_embed(x)
        
        # append mask tokens to sequence
        mask_tokens = self.mask_token.repeat(x.shape[0], ids_restore.shape[1] + 1 - x.shape[1], 1)
        x_ = torch.cat([x[:, 1:, :], mask_tokens], dim = 1) # no cls token
        x_ = torch.gather(x_, dim = 1, index = ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2])) # unshuffle
        x = torch.cat([x[:, :1, :], x_], dim = 1) # append cls token
        
        # add pos embed
        x = x + self.decoder_pos_embed
        
        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)
        
        # predictor projection
        x = self.decoder_pred(x)
        
        # remove cls token
        x = x[:, 1:, :]
        
        return x
    
    def forward_loss(self, imgs, pred, mask, batch_mean):
        """
        imgs: [N, 3, H, W]
        pred: [N, L, p*p*3]
        mask: [N, L], 0 is kept, 1 is removed
        """
        target = self.patchify(imgs)
        if self.norm_pix_loss:
            mean = target.mean(dim = -1, keepdim = True)
            var = target.var(dim = -1, keepdim = True)
            target = (target - mean) / (var + 1.e-6) ** .5
        
        loss = (pred - target) ** 2
        loss = loss.mean(dim = -1) # [N, L], mean loss per patch
        if batch_mean:
            loss = (loss * mask).sum() / mask.sum()
        else:
            loss = (loss * mask).sum(dim = 1) / mask.sum(dim = 1)
        return loss
    
    def forward_feature(self, imgs):
        latent, _, _ = self.forward_encoder(imgs)
        return latent[:, 0]
    
    def forward(self, imgs, mask_ratio = 0.75, batch_mean = True):
        latent, mask, ids_restore = self.forward_encoder(imgs, mask_ratio)
        pred = self.forward_decoder(latent, ids_restore) # [N, L, p*p*3]
        loss = self.forward_loss(imgs, pred, mask, batch_mean)
        return loss, self.unpatchify(pred), mask

Model Build-up

In [18]:
def mae_vit_base_patch16_dec512d8b(**kwargs):
    model = MaskedAutoencoderViT(
        patch_size=16, embed_dim=768, depth=12, num_heads=12,
        decoder_embed_dim=512, decoder_depth=8, decoder_num_heads=16,
        mlp_ratio=4, norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model


def mae_vit_large_patch16_dec512d8b(**kwargs):
    model = MaskedAutoencoderViT(
        patch_size=16, embed_dim=1024, depth=24, num_heads=16,
        decoder_embed_dim=512, decoder_depth=8, decoder_num_heads=16,
        mlp_ratio=4, norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model


def mae_vit_huge_patch14_dec512d8b(**kwargs):
    model = MaskedAutoencoderViT(
        patch_size=14, embed_dim=1280, depth=32, num_heads=16,
        decoder_embed_dim=512, decoder_depth=8, decoder_num_heads=16,
        mlp_ratio=4, norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    return model

In [19]:
# set recommended archs
mae_vit_base_patch16 = mae_vit_base_patch16_dec512d8b  # decoder: 512 dim, 8 blocks
mae_vit_large_patch16 = mae_vit_large_patch16_dec512d8b  # decoder: 512 dim, 8 blocks
mae_vit_huge_patch14 = mae_vit_huge_patch14_dec512d8b  # decoder: 512 dim, 8 blocks

## Pre-train 

In [20]:
def show_image(image, title=''):
    # image is [H, W, 3]
    assert image.shape[2] == 3
    plt.imshow(torch.clip((image * food101_std + food101_mean) * 255, 0, 255).int())
    plt.title(title, fontsize=16)
    plt.axis('off')
    return

In [21]:
def run_one_image(img, model, epoch = None):
    x = torch.tensor(img)

    # make it a batch-like
    x = x.unsqueeze(dim=0)
    x = torch.einsum('nhwc->nchw', x)

    # run MAE
    loss, y, mask = model(x.float(), mask_ratio=0.75)
    #y = model.unpatchify(y)
    y = torch.einsum('nchw->nhwc', y).detach().cpu()

    # visualize the mask
    mask = mask.detach()
    mask = mask.unsqueeze(-1).repeat(1, 1, model.patch_embed.patch_size[0]**2 *3)  # (N, H*W, p*p*3)
    mask = model.unpatchify(mask)  # 1 is removing, 0 is keeping
    mask = torch.einsum('nchw->nhwc', mask).detach().cpu()
    
    x = torch.einsum('nchw->nhwc', x)

    # masked image
    im_masked = x * (1 - mask)

    # MAE reconstruction pasted with visible patches
    im_paste = x * (1 - mask) + y * mask

    # make the plt figure larger
    plt.rcParams['figure.figsize'] = [24, 24]

    plt.subplot(1, 4, 1)
    show_image(x[0], "original")

    plt.subplot(1, 4, 2)
    show_image(im_masked[0], "masked")

    plt.subplot(1, 4, 3)
    show_image(y[0], "reconstruction")

    plt.subplot(1, 4, 4)
    show_image(im_paste[0], "reconstruction + visible")
    
    if epoch is not None:
        if epoch == 0 or (epoch + 1) % 5 == 0:
            plt.savefig('/gpfs/home/ht2413/mae_pretrain/mae_pretrain_reconstruction_' + str(epoch + 1) + '.png')

    #plt.show()

In [ ]:
from torch.optim import Adam, AdamW
from torch.utils.data import DataLoader
import torchvision
import numpy as np
from tqdm import tqdm
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.transforms import ToTensor, Compose, Normalize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128
EPOCHS = 120

#train_dataset = processed_food101['train']
train_loader = DataLoader(filtered_train_dataset, shuffle = True, batch_size = BATCH_SIZE)

#model = MaskedAutoencoderViT(img_size = 224, in_chans = 3, patch_size = 16, depth = 12, decoder_depth = 4).to(device)
model = torch.load('/gpfs/home/ht2413/mae_pretrain/mae_pretrain_480.pt')
optimizer = AdamW(model.parameters(), lr = 5e-5, betas=(0.9, 0.95), weight_decay=0.05)

losses = []

for epoch in range(EPOCHS):
    epoch += 480
    if epoch == 480:
        print('Start training!\n')

    train_losses = []
    correct = 0
    total = 0
    idx = 0
    
    model.to(device)

    model.train()
    for batch_sample in tqdm(train_loader):

        optimizer.zero_grad()
        loss, _, _ = model(batch_sample['pixel_values'].to(device))
        loss.backward()
        train_losses.append(loss.item())
        optimizer.step()

        if idx % 10 == 0:
            print(loss.item())
        idx += 1

    losses += train_losses

    train_loss = float(np.mean(train_losses))
    
    if (epoch + 1) % 10 == 0:
        torch.save(model, '/gpfs/home/ht2413/mae_pretrain/mae_pretrain_' + str(epoch + 1) + '.pt')
    
    print('Epoch: {} \t Average Loss: {:.4f} \n)'.format(
          epoch+1, train_loss
    ))
    
    # plot the reconstruction
    run_one_image(filtered_train_dataset[0]['pixel_values'].permute(1, 2, 0).to('cpu'), model.to('cpu'), epoch)

Start training!



  0%|          | 1/592 [00:01<19:40,  2.00s/it]

0.15036098659038544


  2%|▏         | 11/592 [00:21<19:04,  1.97s/it]

0.13063345849514008


...

 95%|█████████▍| 561/592 [28:41<01:22,  2.66s/it]

0.13078172504901886


 96%|█████████▋| 571/592 [29:03<00:44,  2.13s/it]

0.14167997241020203


 97%|█████████▋| 573/592 [29:08<00:42,  2.24s/it]

1 epoch 19.5 min

1h for ~3 epochs

24h = 24 * 60 min = 1440 min

1 day: 1440 / 19.5 = 73.84 epochs ~= 74 epochs

train for 6 days --> at least 500 epochs